# Time Series Forecasting Backtesting using HyperDrive

## Prerequisites
In order to run this notebook, you need to install AML SDK and its widget extension in your environment by running the following commands in commandline or terminal.  
First, you need to activate your environment by running `activate <your env>` or `source activate <your env>`(on Linux).   
`pip install --upgrade azureml-sdk[notebooks,automl]`  
`jupyter nbextension install --py --user azureml.train.widgets`  
`jupyter nbextension enable --py --user azureml.train.widgets`  

## Set up workspace and experiment

In [3]:
from azureml.core import Workspace, Experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name = 'tsbacktest')

Found the config file in: C:\Users\hlu\TSPerf\prototypes\cross_validation\config.json


## Validate script locally
Configure local, user managed environment

In [12]:
from azureml.core.runconfig import RunConfiguration
run_config_user_managed = RunConfiguration()
run_config_user_managed.environment.python.user_managed_dependencies = True
run_config_user_managed.environment.python.interpreter_path = 'C:/Anaconda/envs/tsperf/python.exe'

In [13]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig(source_directory='./', 
                      script='train_validation.py', 
                      arguments=['--data-folder', 'C:/Users/hlu/TSPerf/prototypes/cross_validation/data/', '--n-estimators', '10', '--min-samples-split', '10'],
                      run_config=run_config_user_managed)
run_local = exp.submit(src)

In [73]:
run_local.get_details()
run_local.get_metrics()

{'average pinball loss': 193.81733289262013}

## Submit a single job to BatchAI

### Configure Batch AI cluster

In [4]:
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException

batchai_cluster_name = "hlutsperfnew"
try:
    compute_target = ComputeTarget(workspace=ws, name = batchai_cluster_name)
    if type(compute_target) is BatchAiCompute:
        print('found compute target {}, just use it.'.format(batchai_cluster_name))
    else:
        print('{} exists but it is not a Batch AI cluster. Please choose a different name.'.format(batchai_cluster_name))
except ComputeTargetException:
    print('creating a new compute target...')
    compute_config = BatchAiCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                                autoscale_enabled=True,
                                                                cluster_min_nodes=0, 
                                                                cluster_max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, batchai_cluster_name, compute_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # Use the 'status' property to get a detailed status for the current cluster. 
    print(compute_target.status.serialize()) 
    
# BatchAiCompute.attach(workspace=ws,
#                       name="hlutsperfnew",
#                       resource_id="/subscriptions/ff18d7a8-962a-406c-858f-49acd23d6c01/resourceGroups/hluamlsdkrg/providers/Microsoft.BatchAI/workspaces/hlutsperf5f5171588c9/clusters/hlutsperf5f5171588c9")

# for ct in ws.compute_targets():
#     print(ct.name, ct.type, ct.provisioning_state)

found compute target hlutsperfnew, just use it.


### Configure Docker environment

In [74]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.core.conda_dependencies import CondaDependencies

env = EnvironmentDefinition()

env.python.user_managed_dependencies = False
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'numpy', 'scikit-garden', 'joblib'],
                                                         python_version='3.6.2')
env.python.conda_dependencies.add_channel('conda-forge')
env.docker.enabled=True

### Create Estimator

In [75]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator

script_folder = './'

script_params = {
    '--data-folder': ws.get_default_datastore().as_mount(),
    '--n-estimators': 10,
    '--min-samples-split': 10
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='train_validation.py',
                environment_definition=env)

### Submit job

In [76]:
run_batchai = exp.submit(config=est)

### Check job status

In [78]:
from azureml.train.widgets import RunDetails
RunDetails(run_batchai).show()

_UserRun()

In [83]:
run_batchai.get_details()
run_batchai.get_metrics()

{'average pinball loss': 193.81733289262013}

## Tune hyper parameter using HyperDrive

In [81]:
from azureml.train.hyperdrive import *
ps = RandomParameterSampling({
    '--min-samples-split': choice(5, 10),
    '--n-estimators': choice(10, 100)
})
htc = HyperDriveRunConfig(estimator=est, 
                          hyperparameter_sampling=ps, 
                          primary_metric_name='average pinball loss', 
                          primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                          max_total_runs=8,
                          max_concurrent_runs=4)
htr = exp.submit(config=htc)

The same input parameter(s) are specified in estimator script params and HyperDrive parameter space. HyperDrive parameter space definition will override duplicate entries in estimator. ['--n-estimators', '--min-samples-split'] is the list of overridden parameter(s).


In [82]:
from azureml.train.widgets import RunDetails
RunDetails(htr).show()

_HyperDrive(widget_settings={'childWidgetDisplay': 'popup'})

In [84]:
best_run = htr.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['Arguments']
print(parameter_values)

['--data-folder', '$AZUREML_DATAREFERENCE_workspacefilestore', '--min-samples-split', '5', '--n-estimators', '10']


## Use custom Docker image
**Note**: This part is not working yet. Will update the this section later.

In [5]:
ds = ws.get_default_datastore()

In [6]:
from azureml.core.runconfig import DataReferenceConfiguration
dr = DataReferenceConfiguration(datastore_name=ds.name, 
                   path_on_datastore='./', 
                   mode='mount',
                   overwrite=True)

In [22]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.runconfig import DataReferenceConfiguration

cd_config = RunConfiguration()
cd_config.target = compute_target.name
cd_config.environment.python.user_managed_dependencies = True
cd_config.environment.python.interpreter_path = '/opt/miniconda/envs/tsperf/bin/python' 

cd_config.environment.docker.enabled = True
cd_config.environment.docker.base_image = ' hluamlwsnew6345184683.azurecr.io/prototypes/cross_validation:v1'
cd_config.data_references = {ds.name: dr}

In [23]:
from azureml.core import ScriptRunConfig
src = ScriptRunConfig('./', './train_validation.py', 
                      arguments=['--data-folder', str(ds.as_mount()), '--n-estimators', '10', '--min-samples-split', '10'],
                      run_config=cd_config)

run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: tsbacktest_1539383533817

Streaming azureml-logs/60_control_log.txt

Streaming log file azureml-logs/60_control_log.txt

Streaming azureml-logs/80_driver_log.txt

/opt/miniconda/envs/tsperf/lib/python3.6/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


The experiment failed. Finalizing run...
Logging experiment finalizing status in history service
Traceback (most recent call last):
  File "azureml-setup/context_manager_injector.py", line 126, in <module>
    execute_with_context(cm_objects, options.invocation)
  File "azureml-setup/context_manager_injector.py", line 70, in execute_with_context
    runpy.run_path(sys.argv[0], globals(), run_name="__main__")
  File "/opt/miniconda/envs/tsperf/lib/python3.6/runpy.py", line 263, in run_path
    pkg_name=pkg_name, script_name=fname)
  File "/opt/miniconda/envs/tsper

{'runId': 'tsbacktest_1539383533817',
 'target': 'hlutsperfnew',
 'status': 'Failed',
 'startTimeUtc': '2018-10-12T22:32:16.137089Z',
 'endTimeUtc': '2018-10-12T22:41:32.144465Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '16d93893-8694-4a8f-8548-cb0ee4dd8624'},
 'runDefinition': {'Script': 'train_validation.py',
  'Arguments': ['--data-folder',
   '$AZUREML_DATAREFERENCE_workspacefilestore',
   '--n-estimators',
   '10',
   '--min-samples-split',
   '10'],
  'Framework': 0,
  'Target': 'hlutsperfnew',
  'DataReferences': {'workspacefilestore': {'DataStoreName': 'workspacefilestore',
    'Mode': 'Mount',
    'PathOnDataStore': './',
    'PathOnCompute': None,
    'Overwrite': True}},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': '/opt/miniconda/envs/tsperf/bin/python',
    'UserManagedDependencies': True,
    'CondaDependencies': {'name': 'project_environment',
     '

In [17]:
import azureml
azureml.core.runconfig.DEFAULT_CPU_IMAGE

'mcr.microsoft.com/azureml/base:0.1.3'

In [17]:
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator

env_custom_docker = EnvironmentDefinition()

env_custom_docker.python.user_managed_dependencies = True
env_custom_docker.python.interpreter_path = '/opt/conda/envs/tsperf/python.exe' 

env_custom_docker.docker.enabled = True
# env_custom_docker.docker.base_image_registry = 'hluamlwsnew6345184683.azurecr.io'
env_custom_docker.docker.base_image = 'hluamlwsnew6345184683.azurecr.io/prototypes/cross_validation:latest'

script_folder = './'

script_params = {
    '--data-folder': ws.get_default_datastore().as_mount(),
    '--n-estimators': 10,
    '--min-samples-split': 10
}

est = Estimator(source_directory=script_folder,
                script_params=script_params,
                compute_target=compute_target,
                use_docker=True,
                entry_script='train_validation.py',
                environment_definition=env_custom_docker)


run_batchai = exp.submit(config=est)

run_batchai.get_details()

TrainingException: {
    "error_details": {
        "correlation": {
            "operation": "bccee4f3-4fa534a7ee234d35",
            "request": "I5sZu/220ic="
        },
        "error": {
            "code": "UserError",
            "debugInfo": {
                "innerException": {
                    "innerException": {
                        "message": "Could not cast or convert from System.String to Microsoft.MachineLearning.Execution.Contracts.ContainerRegistry.",
                        "stackTrace": "   at Newtonsoft.Json.Utilities.ConvertUtils.EnsureTypeAssignable(Object value, Type initialType, Type targetType)\n   at Newtonsoft.Json.Utilities.ConvertUtils.ConvertOrCast(Object initialValue, CultureInfo culture, Type targetType)\n   at Newtonsoft.Json.Serialization.JsonSerializerInternalReader.EnsureType(JsonReader reader, Object value, CultureInfo culture, JsonContract contract, Type targetType)",
                        "type": "System.ArgumentException"
                 